In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook, we show how a **DFA** (Deterministic Finite Automaton) can be minimized by identifying states that are equivalent.

We will strictly adhere to the type definitions established in the previous notebooks.

In [ ]:
import { RecursiveSet, Tuple, Value } from "recursive-set";
import { key } from "./05-DFA-2-RegExp";

## Type Definitions

We use the existing types from our DFA implementation.

* `State`: Atomic state identifier (string or number).
* `DFAState`: A state in our DFA (which itself is a set of NFA states).
* `TransRelDet`: The deterministic transition function $\delta$.
* `DFA`: The object structure.

For the minimization algorithm, we introduce specific types to represent the **Quotient Automaton**:

* **`Pair`**: A tuple $(p, q)$ representing two states being compared.
* **`MinState`**: A state in the minimized DFA. Mathematically, this is an **Equivalence Class** $[q]$, represented as a set of original states $\{p \mid p \approx q\}$.
* **`MinDFA`**: The minimized automaton structure, where $Q$ consists of `MinState`s.

In [ ]:
// === Existing Types ===
type State = string | number;
type Char = string;
type DFAState = RecursiveSet<State>; 
type TransRelDet = Map<string, DFAState>;

type DFA = {
  Q: RecursiveSet<DFAState>;
  Sigma: RecursiveSet<Char>;
  delta: TransRelDet;
  q0: DFAState;
  A: RecursiveSet<DFAState>;
};

// === New Types for Minimization (Mathematical View) ===

type Pair = Tuple<[DFAState, DFAState]>;
type MinState = RecursiveSet<DFAState>;
type MinTransRel = Map<string, MinState>;

type MinDFA = {
    Q: RecursiveSet<MinState>;
    Sigma: RecursiveSet<Char>;
    delta: MinTransRel;
    q0: MinState;
    A: RecursiveSet<MinState>;
}

### Helper Functions

**Arbitrary Element:**
The function `arb(M)` returns an *arbitrary* element from a set `M`.
In our implementation using `recursive-set`, elements are internally sorted by their **hash code** for efficiency. Therefore, this function deterministically returns the element with the smallest hash. While this is not necessarily the "smallest" value (numerically or lexicographically), it ensures a consistent canonical representative for any given set.

**Cartesian Product:**
The function `cartProd(A, B)` computes the Cartesian product $A \times B$. We leverage the built-in, optimized `cartesianProduct` method.

In [ ]:
function arb<T extends Value>(M: RecursiveSet<T>): T {
  if (M.isEmpty()) throw new Error("Error: arb called with empty set!");
  return M.raw[0];
}

function cartProd<S extends Value, T extends Value>(
  A: RecursiveSet<S>, 
  B: RecursiveSet<T>
): RecursiveSet<Tuple<[S, T]>> {
  return A.cartesianProduct(B);
}

### Separation Logic

The function `separate` takes four arguments:
- `Pairs`: A set of pairs known to be separable.
- `States`: The set of all states $Q$.
- `Sigma`: The alphabet $\Sigma$.
- `delta`: The transition map.

It returns a **new** set of separable pairs. Two states $q_1, q_2$ become separable if there exists a character $c$ such that their targets $p_1 = \delta(q_1, c)$ and $p_2 = \delta(q_2, c)$ are already known to be separable.

In [ ]:
function separate(
  Pairs: RecursiveSet<Pair>, 
  States: RecursiveSet<DFAState>, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<Pair> {
  
  const newPairs: Pair[] = [];
  
  const statesArr = States.raw;
  const sigmaArr = Sigma.raw;

  for (const q1 of statesArr) {
    for (const q2 of statesArr) {
      for (const c of sigmaArr) {
        const p1 = delta.get(key(q1, c));
        const p2 = delta.get(key(q2, c));
        
        if (p1 && p2) {
          const targetPair = new Tuple<[DFAState, DFAState]>(p1, p2);
          
          if (Pairs.has(targetPair)) {
            newPairs.push(new Tuple<[DFAState, DFAState]>(q1, q2));
            break;
          }
        }
      }
    }
  }
  return RecursiveSet.fromArray(newPairs);
}

### Equivalence Class Lookup

Given a state `p` and a partition of the states (a set of sets), this function returns the specific set (equivalence class) that contains `p`.

In [ ]:
function findEquivalenceClass(
  p: DFAState, 
  Partition: RecursiveSet<MinState>
): MinState {
  for (const C of Partition) {
    if (C.has(p)) return C;
  }
  throw new Error(`State ${p} not found in partition`);
}

### Computing All Separable Pairs

The function `allSeparable` computes the set of all pairs $(p, q)$ that are **distinguishable**.

It uses a **fixed-point iteration**:
1.  **Base Case:** Start with pairs where one state is accepting and the other is not: $(Q \setminus A) \times A \cup A \times (Q \setminus A)$.
2.  **Iteration:** Repeatedly apply `separate` to find pairs that lead to already separated pairs. Stop when no new pairs are added.

In [ ]:
function allSeparable(
  Q: RecursiveSet<DFAState>, 
  A: RecursiveSet<DFAState>, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<Pair> {
  
  const NonAccepting = Q.difference(A);

  const set1 = cartProd(NonAccepting, A);
  const set2 = cartProd(A, NonAccepting);
  
  let Separable = set1.union(set2);

  while (true) {
    const NewPairs = separate(Separable, Q, Sigma, delta);
    
    if (NewPairs.isSubset(Separable)) {
      return Separable;
    }
    
    Separable = Separable.union(NewPairs);
  }
}

### Reachability Analysis

The function `reachable` filters out states that cannot be reached from the start state $q_0$. Unreachable states are irrelevant for the language accepted by the DFA and must be removed before minimization.

We provide two implementations to illustrate the difference between the algorithmic and the mathematical perspective:

1.  **`reachable` (BFS Algorithm):**
    This implementation uses a standard **Breadth-First Search** (queue-based). It is computationally efficient ($O(V+E)$) and visits each state and edge exactly once. This is the preferred implementation for large automata.

In [ ]:
function reachable(
  q0: DFAState, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<DFAState> {
  
  const visited = new RecursiveSet<DFAState>(q0);
  const queue: DFAState[] = [q0];
  
  let head = 0;
  while(head < queue.length) {
      const p = queue[head++];
      
      for (const c of Sigma) {
          const target = delta.get(key(p, c));
          if (target && !visited.has(target)) {
              visited.add(target);
              queue.push(target);
          }
      }
  }
  return visited;
}

2.  **`reachableFixedPoint` (Mathematical Definition):**
    This implementation directly mirrors the set-theoretical definition found in textbooks. It computes the set of reachable states as a **Fixed-Point Iteration**:
    * Start with $R_0 = \{q_0\}$.
    * Iteratively add all direct successors: $R_{i+1} = R_i \cup \{ \delta(q, c) \mid q \in R_i, c \in \Sigma \}$.
    * Stop when the set stabilizes (i.e., $R_{i+1} \subseteq R_i$).



While both functions produce the exact same result, `reachableFixedPoint` re-evaluates known transitions multiple times, making it slower but easier to prove correct mathematically.

In [ ]:
function reachableFixedPoint(
  q0: DFAState, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<DFAState> {
  
  let Result = new RecursiveSet<DFAState>(q0);
  
  while (true) {
    const NewStates = new RecursiveSet<DFAState>();
    
    for (const p of Result) {
      for (const c of Sigma) {
        const target = delta.get(key(p, c));
        if (target) {
          NewStates.add(target);
        }
      }
    }
    
    if (NewStates.isSubset(Result)) {
      return Result;
    }
    Result = Result.union(NewStates);
  }
}

### The Minimization Algorithm

The `minimize` function executes the complete minimization pipeline. It constructs the **Quotient Automaton** $A / \approx$, where the states are the equivalence classes computed in the previous steps.

The algorithm proceeds as follows:

1.  **Reachability Analysis:**
    Filter out states not reachable from $q_0$, as they do not affect the accepted language.

2.  **Compute Distinguishability:**
    Calculate the set of all separable pairs using the fixed-point iteration `allSeparable`.

3.  **Group Equivalence Classes:**
    Construct the partition of the state space. Two states $p, q$ are grouped into the same class (set) if and only if they are **not** separable.
    $$[q] = \{ p \in Q \mid p \approx q \}$$

4.  **Construct the Quotient Automaton:**
    * **Start State:** The class $[q_0]$ containing the original start state.
    * **Accepting States:** The set of classes $\{ [q] \mid q \in A \}$.
    * **Transitions:** The transition function is defined as $\delta'([q], c) = [\delta(q, c)]$.

**Algorithmic Optimization:**
To compute the new transitions efficiently, we build an **Inverse Index** (a mapping $\text{State} \to \text{Class}$).
Without this index, finding the target class $[\delta(q, c)]$ would require iterating through all classes ($O(|Q|)$). With the index, we can look up the target class in constant time ($O(1)$).

In [ ]:
// Local helper for MinKeys
function minKey(q: MinState, c: Char): string {
  return `${q.toString()},${c}`;
}

function minimize(F: DFA): MinDFA {
  let { Q, Sigma, delta, q0, A } = F;
  
  Q = reachable(q0, Sigma, delta);
  const reachableA = A.intersection(Q);
  
  const Separable = allSeparable(Q, reachableA, Sigma, delta);
  
  const EquivClasses = new RecursiveSet<MinState>();
  const Processed = new RecursiveSet<DFAState>();

  for (const q of Q) {
    if (Processed.has(q)) continue;

    const equivalenceClass = new RecursiveSet<DFAState>();
    
    for (const p of Q) {
      const pairToCheck = new Tuple<[DFAState, DFAState]>(p, q);
      if (!Separable.has(pairToCheck)) {
        equivalenceClass.add(p);
        Processed.add(p);
      }
    }
    EquivClasses.add(equivalenceClass);
  }
  
  let newQ0: MinState | undefined;
  
  const stateToClass = new Map<string, MinState>();
  
  for (const M of EquivClasses) {
      if (M.has(q0)) newQ0 = M;
      
      for (const state of M) {
          stateToClass.set(state.toString(), M);
      }
  }
  
  if (!newQ0) throw new Error("Start state vanished!");

  const newAcceptArr: MinState[] = [];
  for (const M of EquivClasses) {
    const representative = arb(M);
    if (A.has(representative)) {
      newAcceptArr.push(M);
    }
  }
  
  const newDelta: MinTransRel = new Map();
  
  for (const M of EquivClasses) {
    const q = arb(M); 
    for (const c of Sigma) {
      const target = delta.get(key(q, c));
      
      if (target) {
        const targetClass = stateToClass.get(target.toString());
        
        if (targetClass) {
             newDelta.set(minKey(M, c), targetClass);
        }
      }
    }
  }
  
  return {
    Q: EquivClasses,
    Sigma,
    delta: newDelta,
    q0: newQ0,
    A: RecursiveSet.fromArray(newAcceptArr)
  };
}

The notebook `08-Test-Minimize.ipynb` provides a test for the function `minimize`.